In [111]:
import numpy as np
import pandas as pd

In [112]:
def gini_index(bi_split, classes):
    # count all samples in both data-splits
    n_datapoints = float(sum([len(g) for g in bi_split]))
    
    gini = 0.0
    for group in bi_split:
        size = float(len(group))
        if size == 0:
            continue
        
        score = 0.0
        classes_amount = group.iloc[:, -1].value_counts()
        for class_val in classes:
            if len(classes_amount) == 1: continue
            prop = classes_amount[class_val] / size
            score += prop * (1 - prop)
            
        
        # weight the group score by its relative size
        gini += score * (size / n_datapoints)
    
    return gini

In [113]:
def test_split(index, value, df):
    left_split = df[df[index] < value]
    right_split = df[df[index] >= value]
    return left_split, right_split

In [124]:
# creates node with bi_split data
def get_best_split(df, min_size):
    class_values = df.iloc[:, -1].unique()
    b_index, b_value, b_score, b_split= 999, 999, 999, None
    
    for index in range(df.shape[1]-1):
        for i, row in df.iterrows():
            # don't check gini_index for every split-size (too expensive)
            # instead for every min_size'th row
            if i % min_size != 0: continue
            bi_split = test_split(index, row[index], df)
            gini = gini_index(bi_split, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_split = index, row[index], gini, bi_split
    
    return {'index': b_index, 'value': b_value, 'bi_split': b_split}

In [125]:
def to_terminal(df_group):
    return df_group.iloc[:, -1].value_counts().idxmax()

In [126]:
def split(node, max_depth, min_size, depth):
    # print(node['bi_split'])
    left, right = node['bi_split']
    del(node['bi_split'])
    
    # if either left or right df have length 0, then they are both the same class
    # => you can't split them anymore => create terminal-node
    if len(left) == 0 or len(right) == 0:
        node['left'] = node['right'] = to_terminal(left.append(right))
        return
        
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_best_split(left, min_size)
        split(node['left'], max_depth, min_size, depth+1)
    
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_best_split(right, min_size)
        split(node['right'], max_depth, min_size, depth+1)

In [135]:
def build_tree(df_train, max_depth=7, min_size=5):
    root = get_best_split(df_train, min_size)
    split(root, max_depth, min_size, 1)
    return root

In [136]:
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print("{}[{} < {}]".format(depth*' ', node['index'], node['value']))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print("{}, => {}".format(depth*' ', node))

In [137]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [138]:
df = pd.read_csv('banknote.csv', header=None)

In [139]:
df = df.sample(frac=1.0)

In [140]:
df_train = df.iloc[:-125, :]
df_test = df.iloc[-125:, :]

In [141]:
tree = build_tree(df_train, max_depth=5, min_size=5)

In [142]:
size_test = len(df_test)
correct = 0
for i in range(len(df_test)):
    predicted = predict(tree, df_test.iloc[i])
    expected = df_test.iloc[i, -1]
    if predicted == expected:
        correct +=1
    print("Predicted:", predicted, ",", "Expected:", expected)
print("Error-rate:", correct / size_test)

Predicted: 1 , Expected: 1
Predicted: 1 , Expected: 1
Predicted: 1 , Expected: 1
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 1
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 1 , Expected: 1
Predicted: 0 , Expected: 0
Predicted: 1 , Expected: 1
Predicted: 0 , Expected: 0
Predicted: 1 , Expected: 1
Predicted: 1 , Expected: 1
Predicted: 1 , Expected: 1
Predicted: 1 , Expected: 1
Predicted: 0 , Expected: 0
Predicted: 1 , Expected: 1
Predicted: 0 , Expected: 0
Predicted: 1 , Expected: 1
Predicted: 1 , Expected: 1
Predicted: 0 , Expected: 0
Predicted: 1 , Expected: 1
Predicted: 1 , Expected: 1
Predicted: 1 , Expected: 1
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 0 , Expected: 0
Predicted: 1 , Expected: 1
P